In [1]:
import os
import json
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from tqdm import tqdm

In [3]:
# إعداد المسارات
project_root = Path.cwd().parent  
data_path = project_root / "data"
code_path = project_root / "code"
dev_set_file = data_path / "QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl"

In [5]:
# تحميل البيانات
def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    print(f'Loaded {len(data)} records from {input_path}')
    return data

In [10]:
# تحميل النموذج BERT  QA
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# نموذج مدرب مسبقًا على سؤال-وجواب متعدد اللغات (يدعم العربية)
model_name = "mrm8488/bert-multi-cased-finetuned-xquadv1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

C:\Users\sa\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sa\.cache\huggingface\hub\models--mrm8488--bert-multi-cased-finetuned-xquadv1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-multi-cased-finetuned-xquadv1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [12]:
# المعالجة
def get_answer_from_model(question, context):
    try:
        result = qa_pipeline(question=question, context=context)
        return {
            "answer": result["answer"],
            "strt_token_indx": result["start"],
            "end_token_indx": result["end"],
            "rank": 1,
            "score": result["score"]
        }
    except:
        return {
            "answer": "",
            "strt_token_indx": 0,
            "end_token_indx": 0,
            "rank": 1,
            "score": 0
        }


In [14]:
# تطبيق النموذج على البيانات
dataset = load_jsonl(dev_set_file)
all_predictions = {}

for item in tqdm(dataset):
    pq_id = item["pq_id"]
    question = item["question"]
    passage = item["passage"]
    answer_dict = get_answer_from_model(question, passage)
    all_predictions[pq_id] = [answer_dict]


Loaded 163 records from C:\Users\sa\quran-qa-2023-main\Task-B\data\QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl


100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [01:18<00:00,  2.09it/s]


In [16]:
# حفظ النتائج
save_path = data_path / "runs/arabert_qa_results.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_predictions, f, ensure_ascii=False, indent=2)
print("Saved predictions to", save_path)


Saved predictions to C:\Users\sa\quran-qa-2023-main\Task-B\data\runs\arabert_qa_results.json


In [22]:
# التقييم
run_file = save_path
gold_file = data_path / "QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl"

!python QQA23_TaskB_eval.py --run_file "{run_file}" --gold_answers_file "{gold_file}"

Loaded 163 records from C:\Users\sa\quran-qa-2023-main\Task-B\data\QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl
pAP@10 = 0.091
EM@1 = 0.031
F1@1 = 0.111
